---
title: When Winter Rages
description: Exploring the Power and Impact of Blizzards

  - Andrew Blackford[1]

[1] The University of Alabama in Huntsville (UAH)

  - Andrew Blackford (editor)
date: August 7, 2025
execute:
   freeze: true
---

# Run This Notebook

<h3><a href="https://hub.openveda.cloud/hub/user-redirect/git-pull?repo=https://github.com/NASA-IMPACT/veda-docs&urlpath=lab/tree/veda-docs/user-guide/notebooks/stories/derechos.ipynb&branch=main">🚀 Launch in VEDA JupyterHub (requires access)</a></h3>

<h4> To obtain credentials to VEDA Hub, <a href = "https://docs.openveda.cloud/user-guide/scientific-computing/getting-access.html"> follow this link for more information.</a></h4>

<div class="alert alert-block" style="
     background-color: #f8d7da;
     color: #721c24;
     border-left: 4px solid #28a745;
  ">
Disclaimer: it is highly recommended to run a tutorial within NASA VEDA JupyterHub, which already includes functions for processing and visualizing data specific to VEDA stories. Running the tutorial outside of the VEDA JupyterHub may lead to errors, specifically related to EarthData authentication. Additionally, it is recommended to use the Pangeo workspace within the VEDA JupyterHub, since certain packages relevant to this tutorial are already installed. </div>

<h4> If you <strong>do not</strong> have a VEDA Jupyterhub Account you can launch this notebook on your local environment using MyBinder by clicking the icon below.</h4>
<br/>
<a href="https://binder.openveda.cloud/v2/gh/NASA-IMPACT/veda-docs/9c8cdbae92906fb7062b8a0c759dad90e223a4f9?urlpath=lab%2Ftree%2Fuser-guide%2Fnotebooks%2Fstories%2Fderechos.ipynb">
<img src="https://binder.openveda.cloud/badge_logo.svg" alt="Binder" title="A cute binder" width="150"/> </a>

## Environment Setup

In [1]:
# Load libraries
#!pip install -q earthaccess pandas xarray fsspec requests pystac_client

import datetime
import glob
import os

import earthaccess
import pandas as pd
import plotutils as putils
import requests
import xarray as xr
from earthaccess import Store
from pystac_client import Client

In [2]:
# For retrieving data already catalogued in VEDA STAC
STAC_API_URL = "https://openveda.cloud/api/stac"
RASTER_API_URL = "https://openveda.cloud/api/raster"

## Overview

Blizzards are extreme winter weather events defined by the National Weather Service (NWS) as a combination of heavy falling or blowing snow, strong winds sustained at 35 mph or higher, and reduced visibility (less than a quarter of a mile) lasting for at least three contiguous hours. These powerful storms can bring everyday life to a standstill by disrupting travel, damaging infrastructure, and exposing people and animals to dangerous elements.

In the United States, blizzards occur most frequently in the northern Great Plains, but they have been reported in every state except Hawaii. While blizzards are traditionally associated with winter, records show they can happen any time between October and June. Advances in weather monitoring and forecasting have improved our ability to detect and report these events, which has led to an increase in reported blizzard activity over the past several decades. On average, 13 blizzards occur annually across CONUS, but the past few decades have seen this number inflate to 19. The geographic spread of these storms has also widened, with more blizzards occurring outside their historical hotspots. Although the intensity of impact by individual storms has generally decreased due to better forecasting, their economic and societal impacts have risen, with associated losses now exceeding $1 billion per year. Understanding blizzards and preparing for their impacts is critical for communities in regions prone to extreme winter weather.


## Climatology of Blizzards

In the United States, blizzards predominantly affect a region often referred to as "Blizzard Alley”, encompassing parts of North Dakota, South Dakota, Minnesota, Iowa, Nebraska, southeastern Wyoming, and eastern Colorado. The flat terrain and scarcity of natural windbreaks, such as forests or mountains, allow cold Arctic air masses to sweep down from Canada unimpeded, leading to frequent blizzard conditions, far more than any other region in the country.

Several key factors contribute to the high frequency of blizzards in this region:

* Flat Topography: The expansive Great Plains offer little resistance to wind, enabling strong gusts to develop and sustain.
* Lack of Vegetation: The absence of significant tree cover means there are few obstacles to slow down or block wind flow.
* Proximity to Arctic Air Masses: The region's location allows cold air from the Arctic to move southward easily, interacting with moist air masses to produce snow.
* Frequent Storm Systems: Low-pressure systems often develop east of the Rocky Mountains, intensifying as they move into the Plains, bringing heavy snowfall and high winds.

While Blizzard Alley is the most blizzard-prone area, other parts of the United States also experience these severe storms: The Northeastern U.S. is known as a secondary Blizzard Alley, as this region frequently experiences nor'easters—powerful storms that can produce blizzard conditions. Additionally, the Front Range of the Rocky Mountains is susceptible to blizzards due to storm systems intensifying as they descend from higher elevations.


# Example: Blizzard Count (1950-2021)

Pull the blizzard climatology data from from the VEDA STAC catalog and visualize


## Processing steps:
1.) Choose STAC catalog ID and date<br />
2.) Retrieve collection information and items from VEDA STAC catalog<br />
3.) Retrieve item statistics and tiling information<br />
4.) Plot data<br />

Nearest neighbor interpolation of NCEI Storm Events Database confirmed blizzard reports from 1950-2021 in the United States, clipped to the Northern Plains, Midwest, and Northeast.


In [3]:
# TODO: Change collection_ID and date
client_STAC = Client.open(STAC_API_URL)

collection_id = "blizzard-count"
date = "2021-12-31"

results = client_STAC.search(collections=[collection_id], datetime=date)

# ── VEDA Collection Request ─────────────────────────────────────────────────────────────────────────────────────

items = list(results.items())
assert len(items) != 0, "No items found"
item = items[0]
collection = item.get_collection()

# grab the dashboard render block
dashboard_render = collection.extra_fields["renders"]["dashboard"]

assets = dashboard_render["assets"][0]
((vmin, vmax),) = dashboard_render["rescale"]

collection

<CollectionClient id=blizzard-count>

In [4]:
# ── VEDA Tile Request ─────────────────────────────────────────────────────────────────────────────────────
colormap_name = "turbo"

# Build endpoint URL without worrying about trailing slashes
response = requests.get(
    f"{RASTER_API_URL.rstrip('/')}/collections/{collection_id}"
    f"/items/{item.id}/WebMercatorQuad/tilejson.json?"
    f"&assets={assets}"
    f"&color_formula=gamma+r+1.05&colormap_name={colormap_name}"
    f"&rescale={vmin},{vmax}",
)

response.raise_for_status()

tiles = response.json()
print(tiles)

{'tilejson': '2.2.0', 'version': '1.0.0', 'scheme': 'xyz', 'tiles': ['https://openveda.cloud/api/raster/collections/blizzard-count/items/StormEvents_Blizzards_cog_2021-12-31/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?assets=cog_default&color_formula=gamma+r+1.05&colormap_name=turbo&rescale=1%2C50'], 'minzoom': 0, 'maxzoom': 24, 'bounds': [-110.62272262199996, 34.803751156000025, -66.21095309899994, 51.52535394900008], 'center': [-88.41683786049995, 43.164552552500055, 0]}


In [5]:
# Use the new plot_folium_from_VEDA_STAC function
m = putils.plot_folium_from_VEDA_STAC(
    tiles_url_template=tiles["tiles"][0],
    center_coords=[43, -92],
    zoom_level=6,
    rescale=(vmin, vmax),
    colormap_name=colormap_name,
    capitalize_cmap=False,  # to better match VEDA colors and matplotlib colors
    layer_name="Blizzard Count",
    date="(1950-2021)",
    colorbar_caption="count",
    attribution="NCEI Blizzard Count (1950-2021)",
    tile_name="NCEI Blizzard Count (1950-2021)",
    opacity=0.8,
    height="800px",
)

# Display the map
m

## Bomb Cyclogenesis

A defining feature of many nor’easters is bomb cyclogenesis, or "bombogenesis", which occurs when a storm's central low pressure drops by at least 24 millibars in 24 hours, signifying rapid intensification of the system. The process is driven by the clash of cold air from the north and warm, humid air over the Atlantic. As the storm deepens, powerful winds spiral inward towards the central low pressure, enhancing snowfall rates and creating whiteout conditions. Bomb cyclogenesis often results in some of the most impactful nor’easters, which have been historically responsible for bringing record-breaking snowfall and hurricane-force winds to parts of the Northeast. The map shows GOES satellite imagery of one example of a Nor'easter undergoing bomb cyclogenesis off the East Coast of the USA on January 4, 2018.

In [27]:
date = "2018-01-04" 
collection_id = "blizzard-goes-bombogenesis"

# ── VEDA Collection Request ─────────────────────────────────────────────────────────────────────────────────────

results = client_STAC.search(collections=[collection_id], datetime=date)
items = list(results.items())
item = items[0]
collection = item.get_collection()

dashboard_render = collection.extra_fields["renders"]["dashboard"]
assets = dashboard_render["assets"][0]
((vmin, vmax),) = dashboard_render["rescale"]
bidx = dashboard_render.get("bidx", [1, 2, 3])

# ── VEDA Tile Request ─────────────────────────────────────────────────────────────────────────────────────

response = requests.get(
    f"{RASTER_API_URL.rstrip('/')}/collections/{collection_id}"
    f"/items/{item.id}/WebMercatorQuad/tilejson.json?"
    f"&assets={assets}"
    f"&bidx={bidx[0]}&bidx={bidx[1]}&bidx={bidx[2]}"
    f"&rescale={vmin},{vmax}"
    f"&resampling=bilinear"
)
response.raise_for_status()
tiles = response.json()

collection

<CollectionClient id=blizzard-goes-bombogenesis>

In [28]:
# Use the new plot_folium_from_VEDA_STAC function
m = putils.plot_folium_from_VEDA_STAC(
    tiles_url_template=tiles["tiles"][0],
    center_coords=[38, -72],
    zoom_level=5,
    rescale=(vmin, vmax),
    layer_name="GOES TrueColor Imagery",
    date="January 4, 2018",
    attribution="GOES TrueColor Imagery",
    tile_name="GOES TrueColor Imagery",
    opacity=0.8,
    height="800px",
    remove_default_legend=True
    
)

# Display the map
m

## DCVZ

A critical factor in the formation of Denver cyclones is the Denver Convergence Vorticity Zone (DCVZ)—a mesoscale atmospheric feature unique to the area. The DCVZ forms due to easterly upslope winds interacting with the Rocky Mountains, creating a zone of cyclonic rotation just east of Denver. This zone can serve as a focal point for the development or intensification of low pressure systems, often enhancing snowfall near Denver and the eastern Colorado Plains. This enhancement of snowfall often carries eastward as the storm moves into the Great Plains, but typically weakens by the time it reaches the Midwest (though there have been a few notable blizzards in the Midwest that can be attributed to Denver Cyclones).

In [8]:
# TODO: Change collection_ID and date
client_STAC = Client.open(STAC_API_URL)

collection_id = "blizzard-count"
date = "2021-12-31"

results = client_STAC.search(collections=[collection_id], datetime=date)

# ── VEDA Collection Request ─────────────────────────────────────────────────────────────────────────────────────

items = list(results.items())
assert len(items) != 0, "No items found"
item = items[0]
collection = item.get_collection()

# grab the dashboard render block
dashboard_render = collection.extra_fields["renders"]["dashboard"]

assets = dashboard_render["assets"][0]
((vmin, vmax),) = dashboard_render["rescale"]

# ── VEDA Tile Request ─────────────────────────────────────────────────────────────────────────────────────
colormap_name = "turbo"

# Build endpoint URL without worrying about trailing slashes
response = requests.get(
    f"{RASTER_API_URL.rstrip('/')}/collections/{collection_id}"
    f"/items/{item.id}/WebMercatorQuad/tilejson.json?"
    f"&assets={assets}"
    f"&color_formula=gamma+r+1.05&colormap_name={colormap_name}"
    f"&rescale={vmin},{vmax}",
)

response.raise_for_status()

tiles = response.json()

# Use the new plot_folium_from_VEDA_STAC function
m = putils.plot_folium_from_VEDA_STAC(
    tiles_url_template=tiles["tiles"][0],
    center_coords=[40.5, -103],
    zoom_level=7,
    rescale=(vmin, vmax),
    colormap_name=colormap_name,
    capitalize_cmap=False,  # to better match VEDA colors and matplotlib colors
    layer_name="DCVZ Blizzard Count",
    date="(1950-2021)",
    colorbar_caption="count",
    attribution="NCEI Blizzard Count (1950-2021)",
    tile_name="NCEI Blizzard Count (1950-2021)",
    opacity=0.8,
    height="800px",
)

# Display the map
m

## Historic Blizzards

### The Great Appalachian Storm of 1950 (Snow Bowl Blizzard)

The Great Appalachian Blizzard of 1950 is remembered as one of the most extraordinary and impactful winter storms in U.S. history. Occurring over Thanksgiving weekend, from November 24 to 30, the storm wreaked havoc across 22 states, bringing record snowfall, hurricane-force winds, and freezing temperatures. Snowfall totals reached unprecedented levels, with Coburn Creek, West Virginia, reporting 62 inches, and widespread accumulations exceeding 30 inches across parts of Ohio, Pennsylvania, and West Virginia. Accompanying winds gusted up to 110 mph in Concord, New Hampshire, and 160 mph at Mount Washington, causing severe damage and widespread power outages. In total, the storm claimed 353 lives and caused an estimated $66.7 million in damages (equivalent to about $845 million today).

# Example: Total Snowfall (1950 Blizzard)

Pull the blizzard total snowfall data from from the VEDA STAC catalog and visualize


In [37]:
import json
import urllib.parse
import requests
from pystac_client import Client

client_STAC = Client.open(STAC_API_URL)

collection_id = "blizzard-snowfall"
date = "1950-11-26"

results = client_STAC.search(collections=[collection_id], datetime=date)
items = list(results.items())
assert len(items) != 0, "No items found"
item = items[0]
collection = item.get_collection()

dashboard_render = collection.extra_fields["renders"]["dashboard"]
assets = dashboard_render["assets"][0]

# --- RSI categorical colormap ---
veda_cmap_rgba = {
    "6":  [30, 144, 201, 255],  # #1e90c9
    "12": [171, 148, 214, 255], # #ab94d6
    "18": [90, 22, 128, 255],   # #5a1680
    "24": [217, 67, 212, 255],  # #d943d4
}

# Compact JSON → URL-encode for query string
cmap_str = json.dumps(veda_cmap_rgba, separators=(",", ":"))
cmap_q   = urllib.parse.quote_plus(cmap_str)

# ❗ IMPORTANT: don’t override rescale/nodata/bidx here; just let the
# tile server use the raw values + our explicit colormap.
response = requests.get(
    f"{RASTER_API_URL.rstrip('/')}/collections/{collection_id}"
    f"/items/{item.id}/WebMercatorQuad/tilejson.json?"
    f"assets={assets}"
    f"&colormap={cmap_q}"
)

response.raise_for_status()
tiles = response.json()
print(tiles["tiles"][0])


https://openveda.cloud/api/raster/collections/blizzard-snowfall/items/RSI_Snowfall_cog_1950-11-26/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?assets=cog_default&colormap=%7B%226%22%3A%5B30%2C144%2C201%2C255%5D%2C%2212%22%3A%5B171%2C148%2C214%2C255%5D%2C%2218%22%3A%5B90%2C22%2C128%2C255%5D%2C%2224%22%3A%5B217%2C67%2C212%2C255%5D%7D


In [38]:
# Legend colors (HTML only)
rsi_custom_colors = [
    {"color": "#1e90c9", "label": "6"},
    {"color": "#ab94d6", "label": "12"},
    {"color": "#5a1680", "label": "18"},
    {"color": "#d943d4", "label": "24"},
]

m = putils.plot_folium_from_VEDA_STAC(
    tiles_url_template=tiles["tiles"][0],
    center_coords=[41, -85],
    zoom_level=5.5,
    rescale=(6, 24),
    colormap_name="viridis",           # ignored for legend when custom_colors is set
    custom_colors=rsi_custom_colors,   # drives categorical legend
    layer_name="Regional Snowfall Index (RSI) Accumulated Snowfall",
    date=f"{date}T00:00:00Z",
    colorbar_caption="RSI Accumulated Snowfall (inches)",
    attribution="Regional Snowfall Index (RSI) Accumulated Snowfall",
    tile_name="Regional Snowfall Index (RSI) Accumulated Snowfall",
    opacity=0.8,
    height="800px",
    capitalize_cmap=False,
    remove_default_legend=True         # no continuous gradient
)

m


## Historic Blizzards

### The Great Blizzard of ‘78 (Cleveland Superbomb)

The Great Blizzard of ‘78, also known as the "Cleveland Superbomb," remains one of the most catastrophic winter storms in U.S. history, and is one of the worst natural disasters to ever impact Ohio, Indiana, and Michigan. Occurring from January 25 to 27, the storm developed as an intense low-pressure system surged northward from the Gulf of America, colliding with Arctic air on the heels of a small clipper system. This convergence created a rapid drop in atmospheric pressure, with Cleveland recording a barometric pressure of 28.28 inches (958 millibars)—one of the lowest ever observed in the continental United States. The rapid intensification, known as bomb cyclogenesis, led to blizzard conditions of unprecedented severity.

# Example: ERA5 Data (1978 Blizzard)

Pull the ERA5 MSLP data from from the VEDA STAC catalog and visualize


ERA5 reanalysis, produced by the European Centre for Medium-Range Weather Forecasts (ECMWF), is a high-resolution global atmospheric dataset that provides historical climate and weather information from 1940 to the present. It combines observations from satellites, weather stations, and other sources with advanced numerical weather prediction models to generate a consistent, spatially complete representation of past atmospheric conditions. ERA5 offers hourly data at a 0.25 degree spatial resolution with multiple atmospheric variables, including temperature, wind speed, cloud coverage, and pressure, along with uncertainty estimates. It is widely used in climate research, meteorology, and environmental studies.


In [33]:
client_STAC = Client.open(STAC_API_URL)

collection_id = "blizzard-era5-mslp"
date = "1978-01-26"

results = client_STAC.search(collections=[collection_id], datetime=date)

# ── VEDA Collection Request ─────────────────────────────────────────────────────────────────────────────────────

items = list(results.items())
assert len(items) != 0, "No items found"
item = items[0]
collection = item.get_collection()

# grab the dashboard render block
dashboard_render = collection.extra_fields["renders"]["dashboard"]

assets = dashboard_render["assets"][0]
((vmin, vmax),) = dashboard_render["rescale"]

collection

<CollectionClient id=blizzard-era5-mslp>

In [34]:
# ── VEDA Tile Request ─────────────────────────────────────────────────────────────────────────────────────
colormap_name = "nipy_spectral"

# Build endpoint URL without worrying about trailing slashes
response = requests.get(
    f"{RASTER_API_URL.rstrip('/')}/collections/{collection_id}"
    f"/items/{item.id}/WebMercatorQuad/tilejson.json?"
    f"&assets={assets}"
    f"&color_formula=gamma+r+1.05&colormap_name={colormap_name}"
    f"&rescale={vmin},{vmax}",
)

response.raise_for_status()

tiles = response.json()
print(tiles)

{'tilejson': '2.2.0', 'version': '1.0.0', 'scheme': 'xyz', 'tiles': ['https://openveda.cloud/api/raster/collections/blizzard-era5-mslp/items/ERA5_MSLP_cog_1978-01-26T09:00:00Z/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?assets=cog_default&color_formula=gamma+r+1.05&colormap_name=nipy_spectral&rescale=972%2C1040'], 'minzoom': 0, 'maxzoom': 24, 'bounds': [-180.0, -90.0, 180.0, 90.0], 'center': [0.0, 0.0, 0]}


In [37]:
# Use the new plot_folium_from_VEDA_STAC function
m = putils.plot_folium_from_VEDA_STAC(
    tiles_url_template=tiles["tiles"][0],
    center_coords=[40, -80],
    zoom_level=4.5,
    rescale=(vmin, vmax),
    colormap_name=colormap_name,
    capitalize_cmap=False,  # to better match VEDA colors and matplotlib colors
    layer_name="ERA5 Mean Sea Level Pressure",
    date=f"{date}T00:00:00Z",
    colorbar_caption="hPa",
    attribution="ERA5 Mean Sea Level Pressure",
    tile_name="ERA5 Mean Sea Level Pressure",
    opacity=0.8,
    height="800px",
)

# Display the map
m

## Historic Blizzards

### 1991 Halloween Blizzard / The Perfect Storm

The 1991 Halloween Blizzard struck the Upper Midwest from October 31 to November 3, and brought unprecedented snowfall and ice accumulation - particularly affecting Minnesota, Iowa, and Wisconsin. In Minnesota, snowfall began on Halloween morning and intensified over the following days, resulting in record-breaking accumulations. The Twin Cities (Minneapolis-St. Paul) experienced 8.2 inches of snow on October 31, with a storm total of 28.4 inches, marking it as the largest single snowstorm in the city's history. Duluth, Minnesota recorded an astonishing 36.9 inches, and nearby areas along the coast of Lake Superior saw up to 45 inches.

The blizzard's impact was compounded by strong winds, creating snowdrifts up to 10 feet high, which paralyzed transportation and led to widespread power outages. In Iowa, the storm began with a mix of snow, sleet, and freezing rain, resulting in significant ice accumulation that downed power lines and trees, leaving thousands without electricity. The hazardous conditions caused numerous traffic accidents and at least 20 fatalities across the affected regions.


# Example: MERRA-2 Data (1991 Blizzard)

Pull the MERRA-2 10-meter wind gust data from from the VEDA STAC catalog and visualize


The Modern-Era Retrospective Analysis for Research and Applications, Version 2 (MERRA-2) is NASA’s atmospheric reanalysis dataset that spans from 1980 to the present. Produced by the *lobal Modeling and Assimilation Office (GMAO), MERRA-2 improves upon its predecessor by incorporating aerosol interactions, bias corrections for satellite radiances, and observed precipitation to improve the representation of the hydrological cycle within the dataset. It provides global atmospheric, land surface, and oceanic variables such as temperature, wind speed, cloud cover, and pressure at 0.5° x 0.625° spatial resolution with hourly to monthly temporal coverage. MERRA-2 is widely used for climate studies, weather analysis, and Earth system research, particularly for investigating long-term atmospheric trends and variability.


In [44]:
client_STAC = Client.open(STAC_API_URL)

collection_id = "blizzard-merra2-10m-wind"
date = "1991-11-01"

results = client_STAC.search(collections=[collection_id], datetime=date)

# ── VEDA Collection Request ─────────────────────────────────────────────────────────────────────────────────────

items = list(results.items())
assert len(items) != 0, "No items found"
item = items[0]
collection = item.get_collection()

# grab the dashboard render block
dashboard_render = collection.extra_fields["renders"]["dashboard"]

assets = dashboard_render["assets"][0]
((vmin, vmax),) = dashboard_render["rescale"]

collection

<CollectionClient id=blizzard-merra2-10m-wind>

In [48]:
# ── VEDA Tile Request ─────────────────────────────────────────────────────────────────────────────────────

# Build endpoint URL without worrying about trailing slashes
response = requests.get(
    f"{RASTER_API_URL.rstrip('/')}/collections/{collection_id}"
    f"/items/{item.id}/WebMercatorQuad/tilejson.json?"
    f"&assets={assets}"
    f"&color_formula=gamma+r+1.05&colormap_name={colormap_name}"
    f"&rescale={vmin},{vmax}",
)

response.raise_for_status()

tiles = response.json()
print(tiles)

{'tilejson': '2.2.0', 'version': '1.0.0', 'scheme': 'xyz', 'tiles': ['https://openveda.cloud/api/raster/collections/blizzard-merra2-10m-wind/items/MERRA2_10m_Wind_cog_1991-11-01T21:00:00Z/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?assets=cog_default&color_formula=gamma+r+1.05&colormap_name=ylorrd&rescale=0%2C25'], 'minzoom': 0, 'maxzoom': 24, 'bounds': [-180.3125, -90.43922186252138, 179.88109636856717, 90.25], 'center': [-0.21570181571641456, -0.09461093126068931, 0]}


In [50]:
# Use the new plot_folium_from_VEDA_STAC function
m = putils.plot_folium_from_VEDA_STAC(
    tiles_url_template=tiles["tiles"][0],
    center_coords=[40, -90],
    zoom_level=4.5,
    rescale=(vmin, vmax),
    colormap_name="YlOrRd",
    capitalize_cmap=False,  # to better match VEDA colors and matplotlib colors
    layer_name="MERRA-2 10-Meter Wind Gusts",
    date=f"{date}T00:00:00Z",
    colorbar_caption="m/s",
    attribution="MERRA-2 10-Meter Wind Gusts",
    tile_name="MERRA-2 10-Meter Wind Gusts",
    opacity=0.8,
    height="800px",
)

# Display the map
m

## Historic Blizzards

### 1993 Storm of the Century

The 1993 Storm of the Century, also known as the "Superstorm of 1993," was a massive cyclonic storm that impacted a vast portion of the United States from March 12 to 15, 1993. This unprecedented weather event brought record-breaking snowfall, severe weather, and coastal flooding, affecting areas from the Deep South to the Northeast. In the Appalachian Mountains, the storm produced extraordinary snowfall totals. Mount LeConte in Tennessee recorded nearly five feet of snow, while Mount Mitchell in North Carolina saw 49 inches. These heavy snowfalls led to significant disruptions, including impassable roads and widespread power outages.



# Example: MERRA-2 Data (1993 Blizzard)

Pull the MERRA-2 cloud fraction data from from the VEDA STAC catalog and visualize


In [59]:
client_STAC = Client.open(STAC_API_URL)

collection_id = "blizzard-merra2-cfrac"
date = "1993-03-14"

results = client_STAC.search(collections=[collection_id], datetime=date)

# ── VEDA Collection Request ─────────────────────────────────────────────────────────────────────────────────────

items = list(results.items())
assert len(items) != 0, "No items found"
item = items[0]
collection = item.get_collection()

# grab the dashboard render block
dashboard_render = collection.extra_fields["renders"]["dashboard"]

assets = dashboard_render["assets"][0]
((vmin, vmax),) = dashboard_render["rescale"]

# ── VEDA Tile Request ─────────────────────────────────────────────────────────────────────────────────────

# Build endpoint URL without worrying about trailing slashes
response = requests.get(
    f"{RASTER_API_URL.rstrip('/')}/collections/{collection_id}"
    f"/items/{item.id}/WebMercatorQuad/tilejson.json?"
    f"&assets={assets}"
    f"&rescale={vmin},{vmax}",
)

response.raise_for_status()

tiles = response.json()

# Use the new plot_folium_from_VEDA_STAC function
m = putils.plot_folium_from_VEDA_STAC(
    tiles_url_template=tiles["tiles"][0],
    center_coords=[30, -75],
    zoom_level=3.5,
    rescale=(vmin, vmax),
    colormap_name="Greys",
    capitalize_cmap=False,  # to better match VEDA colors and matplotlib colors
    layer_name="MERRA-2 Cloud Fraction",
    date=f"{date}T00:00:00Z",
    colorbar_caption="m/s",
    attribution="MERRA-2 Cloud Fraction",
    tile_name="MERRA-2 Cloud Fraction",
    opacity=0.8,
    height="800px",
)

# Display the map
m

# Clean Up (Optional)


Remove any **.gif** files that were created to save on storage space. Additionally, remove any **core** files that were created if the kernel crashed.

In [60]:
# find all .gif files in the current directory
for gif_path in glob.glob("*.gif"):
    try:
        os.remove(gif_path)
        print(f"Removed {gif_path}")
    except OSError as e:
        print(f"Error removing {gif_path}: {e}")

# find all core files in the current directory
for core_path in glob.glob("core.*"):
    try:
        os.remove(core_path)
        print(f"Removed {core_path}")
    except OSError as e:
        print(f"Error removing {core_path}: {e}")